In [1]:
import os
import pandas as pd
import numpy as np
import abess
import joblib
import networkx as nx
import math

In [2]:
df = pd.read_csv('time_series_covid19_confirmed_global.csv')
new_df = df.groupby('Country/Region').sum()
df = new_df.transpose()
df = df.drop(index=['Lat','Long'])
df = df.diff()
df = df.drop(index=['1/22/20'])

In [3]:
def iscompatible(kset):
    kset_tuple = kset.apply(tuple,axis=1).tolist()
    G=nx.DiGraph()
    G.add_edges_from(kset_tuple)
    result = nx.is_directed_acyclic_graph(G)
    return result

def mostcauses(kset):
    ksetv = kset.Cause.value_counts()
    mostc = ksetv[ksetv==ksetv.max()].index
    return mostc

def pps(i,kset,edges):
    edges = pd.DataFrame(edges, columns=["From", "To"])
    kset_subset = kset.loc[kset['Cause']==i]
    props1 = set(kset_subset.Effect)
    props = props1
    edges_tuple = edges.apply(tuple,axis=1).tolist()
    G=nx.DiGraph()
    G.add_edges_from(edges_tuple)
    for j in props1:
        if G.has_node(j):
            props = props.union(nx.descendants(G,j)) 
    return props

def batchimport(kset,causes,effects):
    for i in causes:
        for j in effects:
            kset.loc[len(kset)] = [i,j]
    return kset

In [4]:
def fit_model(args):
    model, X, y = args
    model.fit(X, y)
    return model

def abessdag_fast (data_in, nu=0, kset=pd.DataFrame(columns=["Cause", "Effect"])
                   , wkset=pd.DataFrame(columns=["Cause", "Effect"]), strategy="DB"):
    data = data_in.copy()
    if (kset.empty) and (wkset.empty):
        if (nu==0):
            data = data_in.copy()
            order = []
            ancestors = data[[data.std().idxmin()]]
            order.append(data.std().idxmin())
            data = data.drop(data.std().idxmin(), axis=1)
            edges = []
            p = data.shape[1]
            for _ in range(p):
                stdlist = []
                ancestors_values = ancestors.values
                for i in data.columns:
                    model = abess.LinearRegression()
                    model.fit(ancestors_values, data[i])
                    ind = np.nonzero(model.coef_)[0]
                    if ancestors.shape[1]==1:
                        res = data[i] - model.predict(ancestors.values)[:,1]
                    else:
                        res = data[i] - model.predict(ancestors_values)   
                    stdlist.append((i, res.std(), ind))
                mini, _, minind = min(stdlist, key=lambda x: x[1])
                order.append(mini)
                for j in minind:
                    edges.append((ancestors.columns[j], mini))
                ancestors = pd.concat((ancestors, data[mini]), axis=1)
                data = data.drop(mini, axis=1)
            edges = pd.DataFrame(edges, columns=["From", "To"])
            return order, edges
        if (nu<0):
            raise ValueError("The threshold can't be negative.")
        if (nu>0):
            n = data.shape[0]
            order = []
            w=(data**2).sum()
            choose_ind = w[w<=w.min()+nu*n].index
            ancestors = data[choose_ind]
            data = data.drop(choose_ind,axis=1)
            order.append(choose_ind.tolist())
            edges = []
    
            if (len(choose_ind)==1):
                stdlist = pd.DataFrame(data=None,columns=['Vertex','loss','ind'])
                models = []
                for i in data.columns:
                    model = abess.LinearRegression()
                    models.append(model)
                results = joblib.Parallel(n_jobs=-1)(
                    joblib.delayed(fit_model)(args)
                    for args in zip(models, [ancestors.values]*len(models), [data[i] for i in data.columns])
                )
                for i, result in enumerate(results):
                    ind = np.nonzero(result.coef_)
                    res = data[data.columns[i]] - result.predict(ancestors.values)[:,1]
                    stdlist.loc[len(stdlist)] = [data.columns[i], (res**2).sum(), ind]
                w = stdlist["loss"]
                choose_ind = w[w<=w.min()+nu*n].index
                vertex_ind = stdlist["Vertex"][choose_ind].tolist()
                order.append(vertex_ind)
                for k in vertex_ind:
                    minind = stdlist.loc[stdlist["Vertex"]==k,"ind"].item()[0]
                    for j in minind:
                        edges.append((ancestors.columns[j], k))
                    ancestors.insert(loc=0 ,column=k, value=data[k])
                    data = data.drop(k, axis=1)
      
            while (data.shape[1]>0):
                stdlist = pd.DataFrame(data=None,columns=['Vertex','loss','ind'])
                models = []
                for i in data.columns:
                    model = abess.LinearRegression()
                    models.append(model)
                results = joblib.Parallel(n_jobs=-1)(
                    joblib.delayed(fit_model)(args)
                    for args in zip(models, [ancestors.values]*len(models), [data[i] for i in data.columns])
                )
                for i, result in enumerate(results):
                    ind = np.nonzero(result.coef_)
                    res = data[data.columns[i]] - result.predict(ancestors.values)
                    stdlist.loc[len(stdlist)] = [data.columns[i], (res**2).sum(), ind]
                w = stdlist["loss"]
                choose_ind = w[w<=w.min()+nu*n].index
                vertex_ind = stdlist["Vertex"][choose_ind].tolist()
                order.append(vertex_ind)
                for k in vertex_ind:
                    minind = stdlist.loc[stdlist["Vertex"]==k,"ind"].item()[0]
                    for j in minind:
                        edges.append((ancestors.columns[j], k))
                    ancestors = pd.concat((ancestors,data[k]),axis=1)
                    data = data.drop(k, axis=1)
            edges = pd.DataFrame(edges, columns=["From", "To"])
            return order, edges
    
    else:
        order = []
        for col in ['Cause', 'Effect']:
            if not set(getattr(kset, col)).issubset(set(data.columns)) or not set(getattr(wkset, col)).issubset(set(data.columns)):
                raise ValueError("Knowledge set should contain the names of variables.")
        if (not iscompatible(kset)) and (not (kset.empty)):
            raise ValueError("Strong knowledge set must be compatible.")
        if (strategy == "KB") and (kset.empty):
            data_frac = data[mostcauses(wkset)]
            ancestors = data_frac[[data_frac.std().idxmin()]]
            order.append(data_frac.std().idxmin())
            wkset = wkset.drop(wkset[wkset['Cause'] == data_frac.std().idxmin()].index)
            data = data.drop(data_frac.std().idxmin(), axis=1)
            edges = []
            p = data.shape[1]
            for _ in range(p):
                stdlist = []
                desc = data.columns if (wkset.empty) else mostcauses(wkset)
                for i in desc:
                    props = pps(i,wkset,edges)
                    props = props.intersection(ancestors.columns)  # Prohibited parent set
                    model = abess.LinearRegression()
                    anc = ancestors.drop(columns=props)
                    if anc.empty:
                        stdlist.append((i, data[i].std(), np.empty((0,), dtype=np.int64)))
                    if anc.shape[1]==1:
                        model.fit(anc.values,data[i])  # Use some of the ancestors to do variable selection
                        ind = np.nonzero(model.coef_)[0]
                        res = data[i] - model.predict(anc.values)[:,1]
                    if anc.shape[1]>=2:
                        model.fit(anc.values,data[i])  # Use some of the ancestors to do variable selection
                        ind = np.nonzero(model.coef_)[0]
                        res = data[i] - model.predict(anc.values)
                    stdlist.append((i, res.std(), ind))
                mini, _, minind = min(stdlist, key=lambda x: x[1])
                order.append(mini)
                props = pps(mini,wkset,edges)
                props = props.intersection(ancestors.columns)
                anc = ancestors.drop(columns=props)
                for j in minind:
                    edges.append((anc.columns[j],mini))
                ancestors = pd.concat((ancestors,data[mini]),axis=1)
                data = data.drop(mini, axis=1)
                wkset = wkset.drop(wkset[wkset['Cause'] == mini].index)
            return order, edges
        else:    
            data_frac = data[data.columns.drop(set(kset.Effect))]
            ancestors = data_frac[[data_frac.std().idxmin()]]
            order.append(data_frac.std().idxmin())
            kset = kset.drop(kset[kset['Cause'] == data_frac.std().idxmin()].index)
            data = data.drop(data_frac.std().idxmin(), axis=1)
            edges = []
            p = data.shape[1]
            for _ in range(p):
                stdlist = []
                cols_to_drop = set(kset.Effect) & set(data.columns)
                for i in data.columns.drop(cols_to_drop):
                    props = pps(i,wkset,edges)
                    props = props.intersection(ancestors.columns)  # Prohibited parent set
                    model = abess.LinearRegression()
                    anc = ancestors.drop(columns=props)
                    if anc.empty:
                        stdlist.append((i, data[i].std(), np.empty((0,), dtype=np.int64)))
                    if anc.shape[1]==1:
                        model.fit(anc.values,data[i])  # Use some of the ancestors to do variable selection
                        ind = np.nonzero(model.coef_)[0]
                        res = data[i] - model.predict(anc.values)[:,1]
                    if anc.shape[1]>=2:
                        model.fit(anc.values,data[i])  # Use some of the ancestors to do variable selection
                        ind = np.nonzero(model.coef_)[0]
                        res = data[i] - model.predict(anc.values)
                    stdlist.append((i, res.std(), ind))
                mini, _, minind = min(stdlist, key=lambda x: x[1])
                order.append(mini)
                props = pps(mini,wkset,edges)
                props = props.intersection(ancestors.columns)
                anc = ancestors.drop(columns=props)
                for j in minind:
                    edges.append((anc.columns[j],mini))
                ancestors = pd.concat((ancestors,data[mini]),axis=1)
                data = data.drop(mini, axis=1)
                kset = kset.drop(kset[kset['Cause'] == mini].index)
                
            return order, edges

In [5]:
zeros_count = df.eq(0).sum()
for col, count in zeros_count.items():
    if count > 570:
        print(f"Column '{col}' has {count} zeros.")

Column 'Andorra' has 623 zeros.
Column 'Antarctica' has 1141 zeros.
Column 'Antigua and Barbuda' has 805 zeros.
Column 'Benin' has 962 zeros.
Column 'Bhutan' has 620 zeros.
Column 'Botswana' has 905 zeros.
Column 'Brunei' has 697 zeros.
Column 'Burkina Faso' has 608 zeros.
Column 'Burundi' has 629 zeros.
Column 'Cameroon' has 937 zeros.
Column 'Central African Republic' has 905 zeros.
Column 'Chad' has 630 zeros.
Column 'Comoros' has 692 zeros.
Column 'Congo (Brazzaville)' has 966 zeros.
Column 'Diamond Princess' has 1127 zeros.
Column 'Dominica' has 940 zeros.
Column 'El Salvador' has 657 zeros.
Column 'Equatorial Guinea' has 834 zeros.
Column 'Eritrea' has 633 zeros.
Column 'Fiji' has 790 zeros.
Column 'Gabon' has 776 zeros.
Column 'Gambia' has 818 zeros.
Column 'Ghana' has 591 zeros.
Column 'Grenada' has 875 zeros.
Column 'Guinea' has 600 zeros.
Column 'Guinea-Bissau' has 737 zeros.
Column 'Holy See' has 1129 zeros.
Column 'Kiribati' has 1052 zeros.
Column 'Korea, North' has 1141 ze

In [6]:
columns_to_drop = zeros_count[zeros_count > 570].index.tolist()
# 从 DataFrame 中删除符合条件的列
df = df.drop(columns_to_drop, axis=1)

In [7]:
window_size = 3
df = df.rolling(window_size).mean()
df = df.drop(index=['1/23/20','1/24/20'])

In [8]:
pop = pd.read_csv('population.csv')
pop = pop.set_index('Country Name').transpose()
pop = pop.squeeze()
sqpop =  pop.apply(np.sqrt)
dfbysqpop = df.div(sqpop, axis=1)

In [9]:
dfbysqpop

Country/Region,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,...,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
1/25/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000068,0.0,0.0,0.00000,0.000000
1/26/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000263,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000
1/27/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000329,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000
1/28/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000329,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000
1/29/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000132,0.000000,0.000000,0.0,...,0.000436,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3/5/23,0.001684,0.003777,0.000702,0.000000,0.000000,0.000000,0.041631,1.647050,0.009736,0.0,...,0.045421,0.000406,0.0,0.000000,0.004080,0.000979,0.0,0.0,0.00423,0.000000
3/6/23,0.002316,0.000000,0.000401,0.000000,0.040976,0.000000,0.000000,1.355555,0.008585,0.0,...,0.030716,0.000000,0.0,0.000000,0.006968,0.000540,0.0,0.0,0.00423,0.000000
3/7/23,0.003527,0.000000,0.001053,0.000000,0.040976,0.000000,0.000000,1.420381,0.010574,0.0,...,0.044332,0.000244,0.0,0.000000,0.006654,0.000810,0.0,0.0,0.00423,0.000000
3/8/23,0.003211,0.003181,0.000853,0.000624,0.040976,0.097569,0.000000,1.662421,0.010260,0.0,...,0.044005,0.003331,0.0,0.009929,0.006654,0.000945,0.0,0.0,0.00000,0.012419


In [10]:
def first_nonzero_index(column):
    nonzero_indices = column.to_numpy().nonzero()[0]
    return nonzero_indices[0] if len(nonzero_indices) > 0 else None

first_nonzero_indices = dfbysqpop.apply(first_nonzero_index)
result = pd.concat([pop,first_nonzero_indices], axis=1)
rowlist = list(set(result[0]))

In [11]:
kset = pd.DataFrame({"Cause":[],"Effect":[]})
for a, b in zip(rowlist, rowlist[1:]):
    kset = batchimport(kset,list(result.loc[result[0] == a].index),list(result.loc[result[0] == b].index))
second_day_nonzero = ['Canada', 'France', 'Korea, South', 'Malaysia', 'Nepal', 'Singapore', 'Taiwan*', 'Thailand', 'Vietnam']
chinaus=['China','US']
kset = batchimport(kset,chinaus,second_day_nonzero)

In [12]:
data = dfbysqpop.iloc[56:206, :]
DAGresult = abessdag_fast(data,kset=kset)

In [33]:
DAGresult[1]

[('China', 'US'),
 ('US', 'Vietnam'),
 ('China', 'Taiwan*'),
 ('US', 'Taiwan*'),
 ('China', 'Thailand'),
 ('Vietnam', 'Thailand'),
 ('Taiwan*', 'Thailand'),
 ('US', 'Korea, South'),
 ('Taiwan*', 'Korea, South'),
 ('US', 'Malaysia'),
 ('Thailand', 'Malaysia'),
 ('Korea, South', 'Nepal'),
 ('Malaysia', 'Nepal'),
 ('China', 'Canada'),
 ('US', 'Canada'),
 ('Taiwan*', 'Canada'),
 ('Malaysia', 'Canada'),
 ('Nepal', 'Canada'),
 ('US', 'Singapore'),
 ('Vietnam', 'Singapore'),
 ('Taiwan*', 'Singapore'),
 ('Thailand', 'Singapore'),
 ('Korea, South', 'Singapore'),
 ('Malaysia', 'Singapore'),
 ('Nepal', 'Singapore'),
 ('Canada', 'Singapore'),
 ('Malaysia', 'France'),
 ('China', 'Australia'),
 ('US', 'Australia'),
 ('Vietnam', 'Australia'),
 ('Taiwan*', 'Australia'),
 ('Nepal', 'Australia'),
 ('Canada', 'Australia'),
 ('US', 'Japan'),
 ('Vietnam', 'Japan'),
 ('Canada', 'Japan'),
 ('Taiwan*', 'Cambodia'),
 ('Australia', 'Cambodia'),
 ('Vietnam', 'Sri Lanka'),
 ('Thailand', 'Germany'),
 ('Canada', 'G

In [13]:
len(DAGresult[0])

140

In [14]:
len(DAGresult[1])

814

In [15]:
cont = pd.read_csv("continent.csv")
countries = list(set([item for sublist in DAGresult[1] for item in sublist]))
cont = cont[["Country Name", "continent"]]
merged_df = pd.merge(pd.DataFrame({"Country Name": countries}), cont, on="Country Name")

In [16]:
america_df = merged_df[merged_df["continent"] == "America"]
america_countries = america_df["Country Name"].tolist()
filtered_list_america = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if c1 in america_countries and c2 in america_countries]
filtered_list_america

[('US', 'Canada'),
 ('US', 'Chile'),
 ('Chile', 'Brazil'),
 ('Canada', 'Mexico'),
 ('Brazil', 'Mexico'),
 ('US', 'Ecuador'),
 ('Chile', 'Argentina'),
 ('Brazil', 'Argentina'),
 ('Ecuador', 'Argentina'),
 ('Chile', 'Panama'),
 ('Ecuador', 'Jamaica'),
 ('Panama', 'Honduras'),
 ('Mexico', 'Bolivia'),
 ('Peru', 'Bolivia'),
 ('Cuba', 'Venezuela'),
 ('Trinidad and Tobago', 'Venezuela'),
 ('Cuba', 'Suriname'),
 ('Bolivia', 'Guatemala'),
 ('Trinidad and Tobago', 'Haiti'),
 ('Paraguay', 'Belize'),
 ('Trinidad and Tobago', 'Belize')]

In [37]:
filtered_list = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if merged_df.loc[merged_df["Country Name"] == c1, "continent"].iloc[0] ==
                 merged_df.loc[merged_df["Country Name"] == c2, "continent"].iloc[0]]

In [38]:
len(filtered_list)

252

In [40]:
america_df = merged_df[merged_df["continent"] == "America"]
america_countries = america_df["Country Name"].tolist()
filtered_list_america = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if c1 in america_countries and c2 in america_countries]
len(filtered_list_america)

21

In [17]:
asia_df = merged_df[merged_df["continent"] == "Asia"]
asia_countries = asia_df["Country Name"].tolist()
filtered_list_asia = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if c1 in asia_countries and c2 in asia_countries]
len(filtered_list_asia)

106

In [18]:
euro_df = merged_df[merged_df["continent"] == "Europe"]
euro_countries = euro_df["Country Name"].tolist()
filtered_list_euro = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if c1 in euro_countries and c2 in euro_countries]
len(filtered_list_euro)

103

In [19]:
africa_df = merged_df[merged_df["continent"] == "Africa"]
africa_countries = africa_df["Country Name"].tolist()
filtered_list_africa = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if c1 in africa_countries and c2 in africa_countries]
len(filtered_list_africa)

21

In [20]:
australia_df = merged_df[merged_df["continent"] == "Australia"]
australia_countries = australia_df["Country Name"].tolist()
filtered_list_australia = [(c1, c2) for (c1, c2) in DAGresult[1]
                 if c1 in australia_countries and c2 in australia_countries]
len(filtered_list_australia)

1

In [52]:
filtered_list_australia

[('Australia', 'New Zealand')]

In [26]:
america = pd.DataFrame(filtered_list_america)
america.to_csv("america.csv")

In [29]:
africa = pd.DataFrame(filtered_list_africa)
africa.to_csv("africa.csv")

In [27]:
asia = pd.DataFrame(filtered_list_asia)
asia.to_csv("asia.csv")

In [28]:
euro = pd.DataFrame(filtered_list_euro)
euro.to_csv("euro.csv")